In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# The chess interface and game loop

### Dependencies

In [ ]:
%pip install python-chess

import chess
import chess.svg
import chess.polyglot
import chess.gaviota
import time
import random
import json
from IPython.display import clear_output, SVG, display, HTML
from typing import Callable, Union

import import_ipynb
import Globals

#### chess.Board.push_legal
Pushes a move to a board's move stack only if it is a valid, legal move. If the move is illegal or invalid, a ValueError is raised and the board state remains unaffected.

###### <b><u>Arguments</u></b>
<b>move</b> <i>(chess.Move)</i> : The move that should be pushed to the move stack.

###### <b><u>Side effects</u></b>
Pushes a move to the board's move stack, assuming the move was valid and legal.

In [ ]:
def push_legal(self, move: chess.Move):
    if move not in self.legal_moves:
        raise ValueError('Illegal move')
    self.push(move)

chess.Board.push_legal = push_legal
del push_legal

---
<br>

### Game class
Lets the user play a game of chess against the AI using a virtual chess board UI. Alternatively, two AIs may play against each other. The game may use any algorithm for the AI(s) to make their moves.
<br><br>

#### \_\_init\_\_
The constructor for the Game class. Gets called whenever a new Game instance is created.
###### __<u>Arguments</u>__
<b>make_move_algorithm_white</b> <i>(Callable, default: None)</i> : The function that the AI controlling the white pieces should use to make a new move, or None if the white pieces are player-controlled.<br>
<b>make_move_algorithm_black</b> <i>(Callable, default: None)</i> : The function that the AI controlling the black pieces should use to make a new move, or None if the white pieces are player-controlled. <br>
<b>search_depth_white</b> <i>(int, default: 3)</i> : The search depth for the white AI's search algorithm. <br>
<b>search_depth_black</b> <i>(int, default: 3)</i> : The search depth for the black AI's search algorithm. <br>
<b>opening_book</b> <i>(str, default: 'Resources/baron30.bin')</i> : The path to the desired opening book the AIs will use, or None if no opening book is desired. <br>
<b>endgame_tablebase_dir</b> <i>(str, default: '/datasets/gaviota')</i> : The path to the directory of the desired endgame tablebase the AIs will use, or None if no endgame tablebase is desired. <br>
###### <b><u>Side effects</u></b>
If neither white nor black is set to be an AI, the game will exit. At least one AI is required to play the game.


In [ ]:
class Game:
    def __init__(
            self,
            make_move_algorithm_white: Callable=None,
            make_move_algorithm_black: Callable=None,
            search_depth_white: int=3,
            search_depth_black: int=3,
            opening_book: str='Resources/baron30.bin',
            endgame_tablebase_dir: str='/datasets/gaviota'
    ):
        self.opening_book = opening_book
        self.endgame_tablebase_dir = endgame_tablebase_dir

        self.test_playthrough = []
        self.problem_playthroughs = {}
        self.wins = []

        self.board = chess.Board()
        self.board.ending = False

        self.move_times = { chess.WHITE: [], chess.BLACK: [] }
        self.make_move = { chess.WHITE: make_move_algorithm_white, chess.BLACK: make_move_algorithm_black }
        self.search_depth = { chess.WHITE: search_depth_white, chess.BLACK: search_depth_black }
        self.is_ai = self.get_ai_state()

        if self.endgame_tablebase_dir is not None:
            self.endgame_tablebase = chess.gaviota.open_tablebase(self.endgame_tablebase_dir)

        # The game requires that at least one player is an AI - it is not a multiplayer chess game
        if not self.is_ai[chess.WHITE] and not self.is_ai[chess.BLACK]:
            raise ValueError('At least one player must be an AI')

#### Game: class variables  

<b>BOARD_DISPLAY_SIZE</b> <i>(int)</i> : The size of the displayed chess board in pixels. <br>

<b>board</b> <i>(chess.Board)</i> : The chess board instance that's used for the game. <br>

<b>make_move</b> <i>(dict<chess.Color, Callable>)</i> : The algorithms the AIs, playing white and/or black, use to make a move. <br>
<b>search_depth</b> <i>(dict<chess.Color, int>)</i> : The search depth for the AI's search algorithm for either color. <br>
<b>is_ai</b> <i>(dict<chess.Color, bool>)</i> : Whether a color is AI-controlled (True) or player-controlled (False). <br>

<b>move_times</b> <i>(dict<chess.Color, list>)</i> : The time the AI(s) take to make their move. <br>
<b>test_playthrough</b> <i>(list<tuple<int, chess.Move>)</i> : The moves that are made during the game as well as their evaluated move scores, for debugging purposes. <br>
<b>problem_playthroughs</b> <i>(dict<str, list>)</i> : A debug move-and-score list (see above) for each test, useful when debugging multiple chess problem tests. <br>
<b>wins</b> <i>(list<chess.Color>)</i> : A list of colors that have won in the previous tests (None in case of a draw).<br>

<b>opening_book</b> <i>(str)</i> : A path to a Polyglot opening book, or None if no opening book is in use. <br>
<b>endgame_tablebase_dir</b> <i>(str)</i> : A path to a Gaviota endgame tablebase directory, or None if no endgame tablebase is in use. <br>
<b>endgame_tablebase</b> <i>(Union[NativeTablebase, PythonTablebase])</i> : The endgame tablebase object, or None if no endgame tablebase is in use. <br>

In [ ]:
# Constants
Game.BOARD_DISPLAY_SIZE = 500

# Board variables
Game.board = None

# AI and player declarations
Game.make_move = None
Game.search_depth = None
Game.is_ai = None

# Testing declarations
Game.move_times = None
Game.test_playthrough = None
Game.problem_playthroughs = None
Game.wins = None

# Opening book and endgame tablebase declarations
Game.opening_book = None
Game.endgame_tablebase_dir = None
Game.endgame_tablebase = None

### play  

Starts the game.
Depending on the input arguments there are different game-states and features:  

__(AI, AI):__  
In this case, the chess board is displayed visually while two AIs, specified by the user in the constructor of the Game-class, play against each other. Therefore making moves based on the specified algorithms, depending on the current game- / board-state.  The AIs take turns.
  
  
__(AI, Player) - (Player, AI):__  
In this case, the chess board is displayed visually while an AI, specified by the user in the constructor of the Game-class, and a human-Player compete against each other. Therefore, the AI is making the best move evaluated by the algorithm based on the current game- / board-state. On the other hand, the player can specifie the move to make via input (If it's valid and legal).  
The player and AI take turns.

###### <b><u>Arguments</u></b>
__debug__ _(bool, default: False)_ : Indicates, whether the game is started in 'debug-mode' or 'normal-mode'. In debug-mode, a certain chess-problem is played by an AI and further information is tracked. In normal mode, a typical game of chess is started.


######  <u>__Side effects__</u> 
- The state of the board in the Game class is changed.  
- The state of is_first_move is changed after the first move.
- The times the AIs take to make a move is tracked and saved.
- The chess board is visually displayed.
- All taken moves are tracked and saved.
- A "Winner-message" is displayed upon the end of the game.


In [ ]:
def play(self, debug: bool=False) -> None:
    while not self.board.is_game_over():
        self.display()
        if self.is_ai[self.board.turn]:
            # Start time
            time_stamp = time.time()

            # Make move
            if not self.make_opening_move():
                color = self.board.turn
                score, move = self.make_move[color](self.board, color, self.search_depth[color], self.endgame_tablebase)
                self.test_playthrough[len(self.board.move_stack)] = (score, move.uci())

            # Add time difference to move times
            time_stamp = time.time() - time_stamp
            self.move_times[self.board.turn].append(time_stamp)
        else:
            self.make_move_human()

        self.board.check_and_set_ending()
        self.display(wait=True)

    winner_message = 'It\'s a draw!' if self.board.outcome() is None or self.board.outcome().winner is None \
        else f'{"White" if self.board.outcome().winner == chess.WHITE else "Black"} is the winner!'
    print(f'The game has ended. {winner_message}\n')
    print(self.board.outcome()) 

Game.play = play
del play

### display
Displays the chess board graphically.

###### __<u>Arguments</u>__
__wait__ _(bool, default: False)_ : Whether or not to wait to clear the output until new output is available to replace it.
__debug__ _(bool)_ : Whether or not the game should display the board in 'debug-mode' or 'normal-mode'.

###### __<u>Side effects</u>__
The current visual output is overwritten.


In [ ]:
def display(self, debug: bool=False) -> None:
    clear_output(wait=not debug)
    try:
        last_move = self.board.peek()
        board_visual = chess.svg.board(self.board, size=self.BOARD_DISPLAY_SIZE, lastmove=last_move, check=self.get_check_square()) 
    except IndexError:
        board_visual = chess.svg.board(self.board, size=self.BOARD_DISPLAY_SIZE)
    display(board_visual)

Game.display = display
del display

### get_check_squares
checks for pieces or squares giving check.
###### __<u>Returns</u>__
__chess.Square__ , _None [if no square sets gives check]_
###### __<u>Side effects</u>__
If get_check_squares returns a value, the display function will indicate the square giving check with a red background.
    

In [ ]:
def get_check_square(self) -> chess.Square:
    if self.board.is_check():
        return self.board.checkers().pop()
    return None
            
Game.get_check_square = get_check_square
del get_check_square

### make_opening_move
Makes a random move from the opening book, if available. If not, no move is made.
###### __<u>Returns</u>__
__bool__ _(True)_: A move from the opening book was made.  
__bool__ _(False)_:  A move from the opening book was not made.

In [ ]:
def make_opening_move(self) -> bool:
    if self.opening_book is None:
        return False

    with chess.polyglot.open_reader(self.opening_book) as reader:
        possible_moves = [ entry.move for entry in reader.find_all(self.board) ]
        if len(possible_moves) > 0:
            move = random.choice(possible_moves)
            self.board.push(move)
            self.test_playthrough[len(self.board.move_stack)] = ('opening library', move.uci())
            return True

    return False

Game.make_opening_move = make_opening_move
del make_opening_move

### get_ai_state
Define the AIs for the game. Dependent on the ai_algorithms given as arguments in game.play()
this functions defines the player state for the different colors. (AI or Player). If the 
"player" is defined as an ai, the dictionary value to the coresponding chess.Color is defined 
as True.
###### __<u>Returns</u>__
__{chess.Color: Bool}__ _(dict)_


In [ ]:
def get_ai_state(self) -> dict:
    return {
        color: self.make_move[color] is not None
        for color in ( chess.WHITE, chess.BLACK )
    }

Game.get_ai_state = get_ai_state
del get_ai_state


### make_move_human
If the "player" is human, this method enables the human user to make a move through the 
input-console.
###### __<u>Side effects</u>__
A move is pushed onto the move_stack and it is the next players turn.

In [ ]:
def make_move_human(self) -> None:
    input_prompt = 'Please input your move: '
    while True:
        try:
            move = chess.Move.from_uci(input(input_prompt))
            self.board.push_legal(move)
            return
        except ValueError:
            input_prompt = 'Illegal move, please try again: '
                
Game.make_move_human = make_move_human
del make_move_human

### save_state
The movestack as well as all the scores are saved in a file, named by the user. The file is stored in the current working-directory. 
This file can then later be loaded to open a game.
       
###### __<u>Side effect</u>__  
Execution is paused until valid filename is given as user input.

In [ ]:
def save_state(self) -> None:
    while True:
        try:
            filename = input('Please input save name:\n') + '.chessgame'
            with open(filename, 'w') as f:
                state = json.dumps(dict(self.test_playthrough))
                f.write(state)
            return
        except OSError:
            print(f' {filename} is not a valid filename. Please try again.')
Game.save_state = save_state
del save_state

### load_state
A saved game is loaded. Therefore, the game will be recreated by loading the move_stack from a saved game.

###### __<u>Side effect</u>__  
Execution is paused until valid file is given as user input.

In [ ]:
def load_game(self) -> None:
    while True:
        try:
            filename = input('What is your save file called?:\n')
            if not filename.endswith('.chessgame'):
                filename += '.chessgame'
                
            with open(filename) as f:
                s = f.read()
                d = json.loads(s)
                self.test_playthrough = list(d.items())
                self.board.clear_stack()
                
                for (_, moves) in self.test_playthrough:
                    self.board.push_legal(chess.Move.from_uci(moves))

            return
        except OSError:
            print(f'There is no such file called {filename} - please try again')
    
Game.load_game = load_game
del load_game

______________________________________________________________

# Debug Section
<br>

**In this section, the Game.play() and Game.display() - Methods will be modified and saved as Game.debug_play() and Game.display_debug()**

The reason is, that a mode for debuging purpose is generally needed if we want to figure out the reason behind bugs or problems. 
Instead of using the tricky python debuger, we developed a method to show only the desired attributes when we need to see them. 
Through this, we can take a deeper look into the Game.play() method while changing the display()-method, so we can see through step by step or move by move.


---

### debug_play
This is the debugger-method for bug-detection. Here, a dictionary of chess-problems is given. The user can choose to present a certain problem as
argument of the method or choose the problem out of given problems while running the "debugger". Furthermore, it is possible to automate the procedure,
if the methods gets executed in a loop. (Example in Test.ipynb) Through this it is possible to run all tests and track all necessary information. If 
automation is not enabled, the information is not saved, due to the user beeing able to access it through the game-object. (If the method is called
twice or more, the information will be overwritten, so be careful to enable automation if you run multiple tests!).
Besides small differences to enable better debugging-opportunities, this function performs like Game.play(). 
###### <u><i>Arguments</u></i>
__chess_problem__ _Callable(String) [Optional] (Default behavior: None)_  
A string associated with a key of the Constants.chess_problems dictionary.  
__automation__ _Callable(Boolean) [Optional] (Default behavior: False)_  
A boolean indicating whether or not this call is part of a automatic test-run consisting of multiple tests.  
###### <u><i>Side effects</u></i>
__automation = False__  
If automation is set to false, the user has to press a button after every move. Therefore we can deside how long we want wait and look at the 
displayed details before the next move is executed.  
__automation = True__  
All tests run without interruption. After all tests ran, details can be accessed through game.problem_playthroughs, a collection of all debug-dictionaries and 
game.wins, a collection of all winners ('white', 'black' or None -> Draw).    
  
The user has to press any key after each move.

In [ ]:
def debug_play(self, chess_problem = None, automation = False) -> None:
    problem_dict = Globals.CHESS_PROBLEMS
    self.test_playthrough.clear() # clear the debug dictionary - this dict may have contained moves from the last playthrough
    if chess_problem is None or chess_problem not in problem_dict:
        problem = input("Please choose chess-problem: \nAJW, \nAF, \nLE, \nMPN, \nEK, \nGBE \nyou can find further information about the given problems in Constants.ipynb\n")
        while problem not in problem_dict:
            clear_output()
            print("\nAJW, \nAF, \nLE, \nMPN, \nEK, \nGBE")
            problem = input("Wrong input, please try again. Type one of the above numbers to initialze the test")
    else:
        problem = chess_problem

    self.board.set_fen(problem_dict[problem])
    
    while not self.board.is_game_over():
        self.display_debug(automation)
        if self.is_ai[self.board.turn]:
            # Start time
            time_stamp = time.time()

            # Make move
            if not self.make_opening_move():
                color = self.board.turn
                score, move = self.make_move[color](self.board, color, self.search_depth[color], self.endgame_tablebase)
                self.test_playthrough[len(self.board.move_stack)] = (score, move.uci())

            # Add time difference to move times
            time_stamp = time.time() - time_stamp
            self.move_times[self.board.turn].append(time_stamp)
        else:
            self.make_move_human()

        self.board.check_and_set_ending()
        
    self.display_debug(True)
    winner_message = 'It\'s a draw!' if self.board.outcome() is None or self.board.outcome().winner is None \
        else f'{"White" if self.board.outcome().winner == chess.WHITE else "Black"} is the winner!'
    print(f'The game has ended. {winner_message}\n')
    print(self.board.outcome())
    if automation:
        self.problem_playthroughs[problem] = self.test_playthrough
        winner = None
        if self.board.outcome() is None or self.board.outcome().winner is None:
            winner = None 
        elif self.board.outcome().winner == chess.WHITE:
            winner = "white"
        else: winner = "black"
            
        self.wins.append(winner)

Game.debug_play = debug_play
del debug_play

### display_debug
This is the display-method used in the debug_play-method. Besides the method waiting for the user to press a button, this method doesn't differ from the display-method.
    
###### <u><i>Arguments</u></i>
__Bool__ _Boolean_  
A boolean to determine if this call is one of multiple-automated tests.

###### <u><i>Side effects</u></i>
__Bool = True__  
The user has to press any key after each move.

In [ ]:
def display_debug(self, automation: bool=False) -> None:
    display(wait=False)
    if not automation:
        if len(self.board.move_stack) != 0:
            print(self.test_playthrough[len(self.board.move_stack)])
        input("press any button to continue")

Game.display_debug = display_debug
del display_debug

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>